# Exploring TileDB-VCF: 
# Notebook 1. Initiation and general operations
2024-12-13 Daniel P. Brink

This is a notebook for getting familiar with the TileDB-VCF python library.

A large chunk of the commands are based on following this [tutorial](https://tiledb-inc.github.io/TileDB-VCF/examples/tutorial_tiledbvcf_basics.html)

Other resources:
- VCF format specification [URL](https://en.wikipedia.org/wiki/Variant_Call_Format)

- The original paper on vcftools and the VCF file format [URL](https://doi.org/10.1093/bioinformatics/btr330)

# Contents
- [1. Installation and initiation](#1-installation-and-initiation)
- [2. Ingestion of data](#2-ingestion-of-data)
- [3. Queries based on metadata](#3-queries-based-on-metadata)
- [4. Export a TileDB-VCF dataset instance to VCF](#4-Export-a-TileDB-VCF-dataset-instance-to-VCF)

# 1. Installation and initiation

Installed TileDB-VCF and other dependencies used in this notebook with conda on a Mac M3:
```
CONDA_SUBDIR=osx-64 conda create -n tileDB_vcf 
conda activate tileDB_vcf
conda install mamba -y
mamba install -c conda-forge libgoogle-cloud=2.26 -y
mamba install -c conda-forge -c bioconda -c tiledb tiledbvcf-py -y
mamba install -c conda-forge -c bioconda -c tiledb libtiledbvcf -y
mamba install jupyter -y
mamba install bcftools -y
```

The `mamba install -c conda-forge libgoogle-cloud=2.26 -y`line was an attempt to circumvent the error, as discussed on the [official forums](https://forum.tiledb.com/t/tiledbvcf-installation-error-on-macos/710) but it did not work on my computer. As is discussed below, this seemed to not be necessary since the error seemed to come from not having loaded all necessary libraries. However, for the sake of reproducibility, the full installation parameters for the conda environment were included above.


In [127]:
import os

import tiledb
import tiledbvcf
import numpy as np
import pandas as pd
import shutil

#Check that Conda and the libraries are installed as expected:
print(f"Current Conda environment: {os.environ['CONDA_DEFAULT_ENV']}")

print(
    f"tiledb v{tiledb.version.version}\n"
    f"numpy v{np.__version__}\n"
    f"tiledb-vcf v{tiledbvcf.version}\n"
)

Current Conda environment: tileDB_vcf
tiledb v0.31.1
numpy v1.26.4
tiledb-vcf v0.34.2



Note! In the official installation documentation for [the TileDB-VCF python package](https://docs.tiledb.com/main/integrations-and-extensions/genomics/population-genomics/installation/quick-install) , it is suggested to run this command in the terminal to verify the install: `python -c "import tiledbvcf; print(tiledbvcf.version)"`. However, as the above code blocks shows, the import works if the tiledb python library is also imported. The corrected command for a user that focuses on tiledbvcf is: `python -c "import tiledb, tiledbvcf; print(tiledbvcf.version)"`

The next step is to setup TileDB’s virtual file system:

In [2]:
vfs = tiledb.VFS(config=tiledb.Config())

# 2. Ingestion of data

Initiate a temp TileDB array

In [99]:
array_uri = "./temp_array_storage/demo-arraylocal"

Check if this array exists from before. If yes, delete the existing array to start fresh.

In [100]:
if (vfs.is_dir(array_uri)):
    print(f"Deleting existing array '{array_uri}'")
    vfs.remove_dir(array_uri)
    print("Done.")

Fetch tutorial data from cloud storage. Create a list of URIs pointing to 5 sample BCF files from S3.

In [101]:
vcf_bucket = "s3://tiledb-inc-demo-data/examples/notebooks/vcfs/1kgp3-chr1"
batch1_samples = ["HG00096.bcf", "HG00097.bcf", "HG00099.bcf", "HG00100.bcf", "HG00101.bcf"]
batch1_uris = [f"{vcf_bucket}/{s}" for s in batch1_samples]
batch1_uris

['s3://tiledb-inc-demo-data/examples/notebooks/vcfs/1kgp3-chr1/HG00096.bcf',
 's3://tiledb-inc-demo-data/examples/notebooks/vcfs/1kgp3-chr1/HG00097.bcf',
 's3://tiledb-inc-demo-data/examples/notebooks/vcfs/1kgp3-chr1/HG00099.bcf',
 's3://tiledb-inc-demo-data/examples/notebooks/vcfs/1kgp3-chr1/HG00100.bcf',
 's3://tiledb-inc-demo-data/examples/notebooks/vcfs/1kgp3-chr1/HG00101.bcf']

Create a TileDB-VCF Dataset instance that opens a connection to our desired array_uri in write mode.

In [102]:
ds = tiledbvcf.Dataset(uri=array_uri, mode="w")
ds
ds.create_dataset(vcf_attrs=batch1_uris[0], enable_allele_count=True, enable_variant_stats=True)

# Verify that the array exists
os.listdir(array_uri)

['__meta',
 'variant_stats',
 'allele_count',
 'sample_stats',
 'data',
 '__tiledb_group.tdb',
 'metadata',
 '__group']

An error message here can imply that the dataset already exists and cannot be overwritten. The if statement above can be run to overcome this.

The dataset instance has now been created, but it does not contain any data yet. The ingest_sample function needs to be run to populate the instance:

In [103]:
%%time
ds.ingest_samples(sample_uris = batch1_uris)

CPU times: user 15.3 s, sys: 4.27 s, total: 19.6 s
Wall time: 2min 37s


This tutorial data is fetched from a S3 cloud. It is likely that a majority of the time required to ingest the data comes from making the remote connection.

Ingest a second list of samples:

In [104]:
%%time 
batch2_samples = ["HG00102.bcf", "HG00103.bcf", "HG00105.bcf", "HG00106.bcf", "HG00107.bcf"]
batch2_uris = [f"{vcf_bucket}/{s}" for s in batch2_samples]
batch2_uris

ds = tiledbvcf.Dataset(uri=array_uri, mode="w") #Incremental update to the array, previous data is not touched 
ds.ingest_samples(sample_uris = batch2_uris)

CPU times: user 15.2 s, sys: 4.12 s, total: 19.3 s
Wall time: 2min 48s


Verify the ingestion by opening the array in read mode and printing out the samples:

In [105]:
ds = tiledbvcf.Dataset(array_uri, mode = "r")
ds.samples()

['HG00096',
 'HG00097',
 'HG00099',
 'HG00100',
 'HG00101',
 'HG00102',
 'HG00103',
 'HG00105',
 'HG00106',
 'HG00107']

So what is the ds object? It is a tiledbvcf.dataset.Dataset, as shown by:

In [106]:
type(ds)

tiledbvcf.dataset.Dataset

The docstring for the `ds` object is through:

In [107]:
help(ds)

Help on Dataset in module tiledbvcf.dataset object:

class Dataset(builtins.object)
 |  Dataset(uri: str, mode: str = 'r', cfg: tiledbvcf.dataset.ReadConfig = None, stats: bool = False, verbose: bool = False, tiledb_config: dict = None)
 |
 |  A class that provides read/write access to a TileDB-VCF dataset.
 |
 |  Parameters
 |  ----------
 |  uri
 |      URI of the dataset.
 |  mode
 |      Mode of operation ('r'|'w')
 |  cfg
 |      TileDB-VCF configuration.
 |  stats
 |      Enable internal TileDB statistics.
 |  verbose
 |      Enable verbose output.
 |  tiledb_config
 |      TileDB configuration, alternative to `cfg.tiledb_config`.
 |
 |  Methods defined here:
 |
 |  __init__(self, uri: str, mode: str = 'r', cfg: tiledbvcf.dataset.ReadConfig = None, stats: bool = False, verbose: bool = False, tiledb_config: dict = None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  attributes(self, attr_type='all') -> list
 |      Return a list of queryable attribut

Some imporant methods of the `ds` object are:

`ds.ingest_samples()`

`ds.samples()`

`ds.read()`

`ds.export()`

We have already seen in the above examples how `ds.ingest_samples` can be used iteratively to add new samples to the dataset instance, and how `ds.samples()` can be used to display all samples in the dataset. Queries with `ds.read()` will be investigated below and in section 3. In section 4, `ds.export` will be used to export the TileDB array to VCF files.

Queries to the instance is done with the `ds.read` command. The output is a pandas dataframe. There are also functions to read the data to a pyarrow table or a Dask dataframe. But let's stick with `ds.read` for now.



In [108]:
%%time
regions = ["1:11106535-11262551"]
samples= ["HG00096"]
attributes = ["sample_name", "contig", "pos_start", "pos_end", "fmt_GT"]

df = ds.read(
    regions = regions,
    samples = samples,
    attrs = attributes,
)
df


CPU times: user 36 ms, sys: 57.3 ms, total: 93.4 ms
Wall time: 113 ms


,sample_name,contig,pos_start,pos_end,fmt_GT
0,HG00096,1,11107439,11107439,"[1, 1]"
1,HG00096,1,11111976,11111979,"[0, 1]"
2,HG00096,1,11112836,11112836,"[0, 1]"
3,HG00096,1,11115299,11115299,"[0, 1]"
4,HG00096,1,11117536,11117536,"[0, 1]"
...,...,...,...,...,...
119,HG00096,1,11254006,11254007,"[0, 1]"
120,HG00096,1,11260848,11260848,"[0, 1]"
121,HG00096,1,11261929,11261929,"[0, 1]"
122,HG00096,1,11262020,11262020,"[0, 1]"


Another attribute that can be included is the Allele Frequency Field (`info_TILEDB_IAF`): 

In [109]:
%%time
regions = ["1:11106535-11262551"]
samples = ["HG00096","HG00097"]
attributes = ["sample_name", "contig", "pos_start", "pos_end", "fmt_GT", "info_TILEDB_IAF"]

df = ds.read(
    regions = regions,
    samples = samples,
    attrs = attributes,
)
df

CPU times: user 51.3 ms, sys: 67.4 ms, total: 119 ms
Wall time: 111 ms


,sample_name,contig,pos_start,pos_end,fmt_GT,info_TILEDB_IAF,alleles
0,HG00096,1,11107439,11107439,"[1, 1]","[0.0, 1.0]","[G, T]"
1,HG00097,1,11107439,11107439,"[1, 1]","[0.0, 1.0]","[G, T]"
2,HG00096,1,11111976,11111979,"[0, 1]","[0.05, 0.95]","[TGAA, T]"
3,HG00097,1,11111976,11111979,"[1, 1]","[0.05, 0.95]","[TGAA, T]"
4,HG00096,1,11112836,11112836,"[0, 1]","[0.05, 0.95]","[T, C]"
...,...,...,...,...,...,...,...
232,HG00097,1,11261929,11261929,"[1, 1]","[0.1, 0.9]","[T, C]"
233,HG00096,1,11262020,11262020,"[0, 1]","[0.1, 0.9]","[A, G]"
234,HG00097,1,11262020,11262020,"[1, 1]","[0.1, 0.9]","[A, G]"
235,HG00096,1,11262310,11262310,"[0, 1]","[0.1, 0.9]","[T, C]"


Allele frequencies can futher be filtered directly in the query with the `set_af_filter` parameter:

In [110]:
df = ds.read(
    regions = regions,
    samples = samples,
    attrs = attributes,
    set_af_filter="<0.5",
)
df

,sample_name,contig,pos_start,pos_end,fmt_GT,info_TILEDB_IAF,alleles
0,HG00096,1,11111976,11111979,"[0, 1]","[0.05, 0.95]","[TGAA, T]"
1,HG00096,1,11112836,11112836,"[0, 1]","[0.05, 0.95]","[T, C]"
2,HG00096,1,11117536,11117536,"[0, 1]","[0.05, 0.95]","[A, T]"
3,HG00096,1,11118390,11118390,"[0, 1]","[0.05, 0.95]","[G, A]"
4,HG00096,1,11119135,11119135,"[0, 1]","[0.05, 0.95]","[G, A]"
...,...,...,...,...,...,...,...
82,HG00096,1,11252805,11252806,"[0, 1]","[0.05, 0.95]","[TA, T]"
83,HG00096,1,11254006,11254007,"[0, 1]","[0.1, 0.9]","[AC, A]"
84,HG00096,1,11261929,11261929,"[0, 1]","[0.1, 0.9]","[T, C]"
85,HG00096,1,11262020,11262020,"[0, 1]","[0.1, 0.9]","[A, G]"


# 3. Queries based on metadata

Metadata-based queries are supported by TileDB-VCF. Documentation can be found in this tutorial:
https://documentation.cloud.tiledb.com/academy/structure/life-sciences/population-genomics/tutorials/advanced/sample-metadata/

This tutorial is based on having access to a dataset on tileDB cloud, which requires a licence. But for testing the code, we can actually use any dataset and make mock metadata for it

First, reinitalize the ds object to avoid buffer errors related to the allele filtering.

In [112]:
ds = tiledbvcf.Dataset(array_uri, mode = "r")
ds.samples()

['HG00096',
 'HG00097',
 'HG00099',
 'HG00100',
 'HG00101',
 'HG00102',
 'HG00103',
 'HG00105',
 'HG00106',
 'HG00107']

Slice out the last four samples from the pandas dataframe:

In [113]:
query_samples = ds.samples()[-4:]
print(query_samples)

['HG00103', 'HG00105', 'HG00106', 'HG00107']


Create mock metadata fields for the dataset. Run it through a function from the tutorial to convert the metadata dataframe to a TileDB array. 

In [116]:
metadata_df = pd.DataFrame(
    {
        "Sample": query_samples,
        "Parent": ["X400", "X400", "X301", "X203"], 
        "Weight": [45, 23, 11, 57], 
        "PureBreed": [True, True, False, True],
    },
)

metadata_uri = "temp_array_storage/mock_metadata_array"

def convert(metadata_df,metadata_uri):
    """Convert pandas.DataFrame to TileDB array.
    
    Delete array if it exists.
    """
    
    try:
        tiledb.from_pandas(
            metadata_uri, 
            metadata_df, 
            index_dims=["Sample"],
        )
    except tiledb.TileDBError:
        print("Array already exists, refreshing.")
        if os.path.exists(metadata_uri):
            shutil.rmtree(metadata_uri)
            
        convert(metadata_df,metadata_uri)
        
convert(metadata_df,metadata_uri)

Array already exists, refreshing.


We can view the schema of the resulting array with:

In [117]:
with tiledb.open(metadata_uri, "r") as Ar:
    
    print(Ar.schema)

ArraySchema(
  domain=Domain(*[
    Dim(name='Sample', domain=('', ''), tile=None, dtype='|S0', var=True, filters=FilterList([ZstdFilter(level=-1), ])),
  ]),
  attrs=[
    Attr(name='Parent', dtype='<U0', var=True, nullable=False, enum_label=None, filters=FilterList([ZstdFilter(level=-1), ])),
    Attr(name='Weight', dtype='int64', var=False, nullable=False, enum_label=None, filters=FilterList([ZstdFilter(level=-1), ])),
    Attr(name='PureBreed', dtype='bool', var=False, nullable=False, enum_label=None, filters=FilterList([ZstdFilter(level=-1), ])),
  ],
  cell_order='row-major',
  tile_order='row-major',
  capacity=10000,
  sparse=True,
  allows_duplicates=True,
)



Now we can query the metadata array for all samples that contain patient "X400":

In [119]:
# example parent we'd like to get samples for
query_parent = "X400"

with tiledb.open(metadata_uri, "r") as Ar:

    # query metadata array for 
    sample_result = Ar.query(cond=f"Parent == '{query_parent}'")[:]

samples = [s.decode() for s in sample_result["Sample"]]
print(samples)

['HG00103', 'HG00105']


Turns out that there were two samples that fulfilled this criterion. We can now pass this query result onwards to a query for the main dataset instance:

In [120]:


variant_results = ds.read(
#    regions=["1:10177-249240219"],
    samples=samples,
    attrs=[
        "sample_name",
        "contig",
        "pos_start",
        "pos_end",
        "alleles",
        "fmt_GT",
    ],
)

variant_results

,sample_name,contig,pos_start,pos_end,alleles,fmt_GT
0,HG00103,1,10177,10177,"[A, AC]","[1, 0]"
1,HG00105,1,10177,10177,"[A, AC]","[1, 0]"
2,HG00103,1,10352,10352,"[T, TA]","[1, 0]"
3,HG00103,1,10616,10637,"[CCGCCGTTGCAAAGGCGCGCCG, C]","[1, 1]"
4,HG00105,1,10616,10637,"[CCGCCGTTGCAAAGGCGCGCCG, C]","[1, 1]"
...,...,...,...,...,...,...
644002,HG00105,1,249240219,249240219,"[A, T]","[1, 0]"
644003,HG00103,1,249240539,249240539,"[T, G]","[1, 0]"
644004,HG00105,1,249240539,249240539,"[T, G]","[0, 1]"
644005,HG00103,1,249240543,249240545,"[AGG, A]","[0, 1]"


To demonstrate the work-flow in a denser presentation, we can put together it to a single code block. For fun, let's include all ten samples and not just the last four. This means that the metadata dataframe need to be updated with values for all samples. Down the line, this could be done by loading e.g. a tabular file, but for now, let's hard code this into the dataframe:

In [121]:
query_samples = ds.samples()
metadata_df = pd.DataFrame(
    {
        "Sample": query_samples,
        "Parent": ["X200","X200","X205","X206","X115","X400","X400","X400", "X301", "X203"], 
        "Weight": [14, 23, 62, 31, 42, 19, 45, 23, 11, 57], 
        "PureBreed": [False, True, True, False, False, False, True, True, False, True],
    },
)
convert(metadata_df,metadata_uri)

query_parent = "X400"

with tiledb.open(metadata_uri, "r") as Ar:
    sample_result = Ar.query(cond=f"Parent == '{query_parent}'")[:]

samples = [s.decode() for s in sample_result["Sample"]]
print(samples)
variant_results = ds.read(
#    regions=["1:10177-249240219"],
    samples=samples,
    attrs=[
        "sample_name",
        "contig",
        "pos_start",
        "pos_end",
        "alleles",
        "fmt_GT",
    ],
)

variant_results

Array already exists, refreshing.
['HG00102', 'HG00103', 'HG00105']


,sample_name,contig,pos_start,pos_end,alleles,fmt_GT
0,HG00102,1,10177,10177,"[A, AC]","[1, 0]"
1,HG00103,1,10177,10177,"[A, AC]","[1, 0]"
2,HG00105,1,10177,10177,"[A, AC]","[1, 0]"
3,HG00102,1,10352,10352,"[T, TA]","[1, 0]"
4,HG00103,1,10352,10352,"[T, TA]","[1, 0]"
...,...,...,...,...,...,...
966388,HG00103,1,249240539,249240539,"[T, G]","[1, 0]"
966389,HG00105,1,249240539,249240539,"[T, G]","[0, 1]"
966390,HG00102,1,249240543,249240545,"[AGG, A]","[1, 1]"
966391,HG00103,1,249240543,249240545,"[AGG, A]","[0, 1]"


# 4. Export a TileDB-VCF dataset instance to VCF

The contents of the TileDB arrays can be written to file using `ds.export`. Let's first investigate how to export the data to a VCF file:

In [122]:
%%time
ds.export(
    regions = ['1:10177-24924'],
    samples = ['HG00101'],
    output_format = 'v',
    output_dir = 'temp_VCF_exports/'
)

CPU times: user 55.8 ms, sys: 100 ms, total: 156 ms
Wall time: 154 ms


Notice that it is possible to include slicing as part of the export option, such as genomic coordinates and samples. VCF files are plain files and can be opened in any editor. In the below example, the file is opened by `less`

In [124]:
!less ./temp_VCF_exports/HG00101.vcf | head -n 50

##fileformat=VCFv4.1
##FILTER=<ID=PASS,Description="All filters passed">
##fileDate=20150218
##reference=ftp://ftp.1000genomes.ebi.ac.uk//vol1/ftp/technical/reference/phase2_reference_assembly_sequence/hs37d5.fa.gz
##source=1000GenomesPhase3Pipeline
##contig=<ID=1,assembly=b37,length=249250621>
##contig=<ID=2,assembly=b37,length=243199373>
##contig=<ID=3,assembly=b37,length=198022430>
##contig=<ID=4,assembly=b37,length=191154276>
##contig=<ID=5,assembly=b37,length=180915260>
##contig=<ID=6,assembly=b37,length=171115067>
##contig=<ID=7,assembly=b37,length=159138663>
##contig=<ID=8,assembly=b37,length=146364022>
##contig=<ID=9,assembly=b37,length=141213431>
##contig=<ID=10,assembly=b37,length=135534747>
##contig=<ID=11,assembly=b37,length=135006516>
##contig=<ID=12,assembly=b37,length=133851895>
##contig=<ID=13,assembly=b37,length=115169878>
##contig=<ID=14,assembly=b37,length=107349540>
##contig=<ID=15,assembly=b37,length=102531392>
##contig=<ID=16,assembly=b37,length=90354753>
##contig

The possible export methods are described in the docstring below. 

The supported output formats are `'b': bcf (compressed), 'u': bcf, 'z':vcf.gz, 'v': vcf.`

In [92]:
help(ds.export)

Help on method export in module tiledbvcf.dataset:

export(samples: (<class 'str'>, typing.List[str]) = None, regions: (<class 'str'>, typing.List[str]) = None, samples_file: str = None, bed_file: str = None, skip_check_samples: bool = False, enable_progress_estimation: bool = False, merge: bool = False, output_format: str = 'z', output_path: str = '', output_dir: str = '.') method of tiledbvcf.dataset.Dataset instance
    Exports data to multiple VCF files or a combined VCF file.

    Parameters
    ----------
    samples
        Sample names to be read.
    regions
        Genomic regions to be read.
    samples_file
        URI of file containing sample names to be read, one per line.
    bed_file
        URI of a BED file of genomic regions to be read.
    skip_check_samples
        Skip checking if the samples in `samples_file` exist in the dataset.
    set_af_filter
        Filter variants by internal allele frequency. For example, to include
        variants with AF > 0.1, set t

VCF files can contain data from one sample, or data from multiple samples. Combining samples can for instance be done with:

In [125]:
%%time
ds.export(
    regions = ['1:10177-24924'],
    samples = ds.samples()[0:5],
    merge = True,
    output_format = 'v',
    output_path = 'temp_VCF_exports/combined.vcf',
)

CPU times: user 52.8 ms, sys: 95.3 ms, total: 148 ms
Wall time: 148 ms


For the sake of variety, let's use `bcftools` to view this file. Calling on this or `vcftools` will be useful to perform benchmarking later on.

In [126]:
!bcftools view ./temp_VCF_exports/combined.vcf | head -1000

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##fileDate=20150218
##reference=ftp://ftp.1000genomes.ebi.ac.uk//vol1/ftp/technical/reference/phase2_reference_assembly_sequence/hs37d5.fa.gz
##source=1000GenomesPhase3Pipeline
##contig=<ID=1,assembly=b37,length=249250621>
##contig=<ID=2,assembly=b37,length=243199373>
##contig=<ID=3,assembly=b37,length=198022430>
##contig=<ID=4,assembly=b37,length=191154276>
##contig=<ID=5,assembly=b37,length=180915260>
##contig=<ID=6,assembly=b37,length=171115067>
##contig=<ID=7,assembly=b37,length=159138663>
##contig=<ID=8,assembly=b37,length=146364022>
##contig=<ID=9,assembly=b37,length=141213431>
##contig=<ID=10,assembly=b37,length=135534747>
##contig=<ID=11,assembly=b37,length=135006516>
##contig=<ID=12,assembly=b37,length=133851895>
##contig=<ID=13,assembly=b37,length=115169878>
##contig=<ID=14,assembly=b37,length=107349540>
##contig=<ID=15,assembly=b37,length=102531392>
##contig=<ID=16,assembly=b37,length=90354753>
##contig